In [ ]:
import torch
import cv2              # OpenCV库，用于读取图片
import os               # 系统库，用于处理路径
import numpy as np      # 数学库，用于处理数组（图像本质就是数组）
from torch.utils.data import Dataset # PyTorch标准数据集工具（这段代码里虽没用到，但通常都会导入）
import matplotlib.pyplot as plt      # 画图工具（这里没用到，通常用于本地弹窗看图）
from torch.utils.tensorboard import SummaryWriter # 导入TensorBoard的核心记录工具
from torchvision import transforms  # 导入图像变换工具  

img_path = r'data\train\ants_image\5650366_e22b7e1065.jpg'
img = cv2.imread(img_path)